In [1]:
import cv2
import numpy as np

In [5]:
def cosine_similarity(x, y):
    numerator = 0
    denominator = 0

    # *************** your code here *******************
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)
    return numerator / denominator

In [6]:

def computeCorrelationCofficient(X, Y):
  N = len(X)
  numerator = 0
  denominator = 0
  #your code here ******************
  prod=[]
  for xi, yi in zip(X, Y):
    prod.append(xi*yi)
  sumxy = sum(prod)
  sumx = sum(X)
  sumy = sum(Y)

  squared_sum_x = sumx**2
  squared_sum_y = sumy**2

  x_squared = []
  for xi in X:
    x_squared.append(xi**2)
  sum_x_squared = sum(x_squared)

  y_squared = []
  for yi in Y:
    y_squared.append(yi**2)
  sum_y_squared = sum(y_squared)

  numerator = N*sumxy - sumx*sumy
  denominator1 = N*sum_x_squared - squared_sum_x
  denominator2 = N*sum_y_squared - squared_sum_y
  denominator = (denominator1*denominator2)**0.5

  return np.round(numerator / denominator,2)

In [7]:
def window_based_matching_cosine(left_img, right_img, disparity_range, kernel_size=5):
    # Read left, right images then convert to grayscale
    left  = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left  = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Create blank disparity map
    depth = np.zeros((height, width), np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, height-kernel_half):
        print("\rProcessing.. %d%% complete" % (y/(height-kernel_half)*100),end="",flush=True)
        for x in range(kernel_half, width-kernel_half):
            # Find j where cost has maximum value
            disparity = 0
            cost_optimal  = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    wp = left[(y-kernel_half):(y+kernel_half)+1, (x-kernel_half):(x+kernel_half)+1]
                    wqd = right[(y-kernel_half):(y+kernel_half)+1, (d-kernel_half):(d+kernel_half)+1]

                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd.flatten()

                    cost = cosine_similarity(wp_flattened, wqd_flattened)

                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j

                 # *************** your code here *******************

            # Let depth at (y, x) = j (disparity)
            # Multiply by a scale factor for visualization purpose
            depth[y, x] = disparity * scale
    return depth

In [4]:


left_img_path = "MiddleburyDataset/normal condition/view1.png"
right_img_path = "MiddleburyDataset/normal condition/view5.png"

disparity_range = 64
kernel_size = 5

left = cv2.imread(left_img_path)
right = cv2.imread(right_img_path)

cv2.imshow('left',left)
cv2.imshow('right',right)
cv2.waitKey(0)

-1

In [9]:
left_img_path = "MiddleburyDataset/normal condition/view1.png"
right_img_path = "MiddleburyDataset/normal condition/view5.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_cosine(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)
cv2.imshow('depth',depth)
cv2.imshow('depth2',cv2.applyColorMap(depth, cv2.COLORMAP_JET))
cv2.waitKey(0)

Processing.. 0% complete

Processing.. 99% complete

-1

In [10]:
#Question 5

def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "MiddleburyDataset/normal condition/view1.png"
right_img_path = "MiddleburyDataset/normal condition/view5.png"
groundtruth_img_path = "MiddleburyDataset/disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_cosine(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

Processing.. 0% complete

Processing.. 99% completermse : 4.98612760469305


In [11]:
def window_based_matching_correlation(left_img, right_img, disparity_range, kernel_size=5):
    # Read left, right images then convert to grayscale
    left  = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left  = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Create blank disparity map
    depth = np.zeros((height, width), np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, height-kernel_half):
        print("\rProcessing.. %d%% complete" % (y/(height-kernel_half)*100),end="",flush=True)
        for x in range(kernel_half, width-kernel_half):
            # Find j where cost has maximum value
            disparity = 0
            cost_optimal  = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    wp = left[(y-kernel_half):(y+kernel_half)+1, (x-kernel_half):(x+kernel_half)+1]
                    wqd = right[(y-kernel_half):(y+kernel_half)+1, (d-kernel_half):(d+kernel_half)+1]

                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd.flatten()

                    cost = computeCorrelationCofficient(wp_flattened, wqd_flattened)

                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j
              # *************** your code here *******************

            # Let depth at (y, x) = j (disparity)
            # Multiply by a scale factor for visualization purpose
            depth[y, x] = disparity * scale
    return depth

In [12]:
#Question 6
def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "MiddleburyDataset/normal condition/view1.png"
right_img_path = "MiddleburyDataset/normal condition/view5.png"
groundtruth_img_path = "MiddleburyDataset/disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_correlation(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

Processing.. 58% complete

C:\Users\vomin\AppData\Local\Temp\ipykernel_11408\2873938609.py:31: RuntimeWarning: invalid value encountered in scalar divide
  return np.round(numerator / denominator,2)


Processing.. 99% completermse : 5.359850329069782


In [13]:
#Question 7

def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "MiddleburyDataset/dificult condition/view1_d.png"
right_img_path = "MiddleburyDataset/dificult condition/view5_d.png"
groundtruth_img_path = "MiddleburyDataset/disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_cosine(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

Processing.. 99% completermse : 5.605343375188772


In [14]:
#Question 8

def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "MiddleburyDataset/dificult condition/view1_d.png"
right_img_path = "MiddleburyDataset/dificult condition/view5_d.png"
groundtruth_img_path = "MiddleburyDataset/disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_correlation(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

Processing.. 7% complete

C:\Users\vomin\AppData\Local\Temp\ipykernel_11408\2873938609.py:31: RuntimeWarning: invalid value encountered in scalar divide
  return np.round(numerator / denominator,2)


Processing.. 99% completermse : 5.38587686965895


In [15]:
cv2.imshow('depth',cv2.applyColorMap(depth, cv2.COLORMAP_JET))
cv2.waitKey(0)

-1